# Projet Text Mining

## Librairies

In [184]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [185]:
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#import contractions
import nltk 
import time
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [186]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
from nltk.stem import WordNetLemmatizer
wnl=WordNetLemmatizer()

## Importation des datas

In [187]:
df = pd.read_csv('Data/dataset.csv', sep='\t', header = None, names = ["Avis"], nrows=20)
labels = pd.read_csv('Data/labels.csv', sep='\t', header = None, names = ['Note'])
#sns.heatmap(df.isnull(), cbar=False)
all = pd.concat([df.Avis,labels.Note], sort =True)

display(df)

,Avis
0,Obviously made to show famous 1950s stripper M...
1,This film was more effective in persuading me ...
2,Unless you are already familiar with the pop s...
3,From around the time Europe began fighting Wor...
4,Im not surprised that even cowgirls get the bl...
5,(48 out of 278 people found this comment usefu...
6,Went to watch this movie expecting a 'nothing ...
7,A good cast and they do their best with what t...
8,The only thing that kept me from vomiting afte...
9,"I just watched this film 15 minutes ago, and I..."


# Etape 1 : Rechercher les meilleurs pré-traitements et les meilleurs classifieurs

Dans cette étape vous pourrez effectuer différents prétraitements et utiliser différents classifieurs
afin de rechercher ceux qui sont les plus efficaces. Il ne faut pas hésiter à utiliser différents
classifieurs car un classifieur qui s’avère efficace sur un jeu de données est peut être inefficace sur
un autre jeu de données. Attention également à l’évaluation de vos modèles.

### 2) Nettoyage des données

In [188]:
dfc=df.Avis.copy()
dfc_phrases = df.Avis.copy()

debut = time.time()
for i in range(0, len(df), 1):
    
    dfc[i]=''.join(dfc[i]).lower() #jusqu'ici c'est une liste
    dfc[i]=sent_tokenize(dfc[i])# ici cast de la liste en string(avis entier)
    dfc_phrases[i] = dfc[i] # Ici on va chuck nos phrase et apposer nos tag

    for j in range(0, len(dfc.loc[i]), 1):
        
        #dfc.loc[i][j]= contractions.fix(dfc.loc[i][j])
        dfc.loc[i][j]=dfc.loc[i][j].lower()
        dfc.loc[i][j]=word_tokenize(dfc.loc[i][j])
        sentence = dfc.loc[i][j]
        sentence_without_sw = sentence# creation d'une liste de phrase tmp
        for word in sentence:
            #word = nltk.pos_tag(wnl.lemmatize(word, pos='v'))
            for sw in stop_words:
                if word==sw or word=="." or word==":" or word=="," or word==";" or word==')' or word =='(':
                    if word in sentence_without_sw:
                        sentence_without_sw.remove(word)
        sentence = sentence_without_sw #permet de retirer les stop word sans changer la longueur de la liste
fin = time.time()
print("Le temps écoulé pour ce traitement a prit : ", fin-debut)
display(dfc_phrases[19]) 
#1 ere ligne bien parsé mais pas taggé => ['obviously', 'made', 'show', 'famous', '1950s', 'stripper',
#'misty', 'ayers', '``', 'acting', "''", 'talents']


Le temps écoulé pour ce traitement a prit :  0.6100101470947266


[['mediocre', 'sci-fi', 'channel', 'original', 'picture'],
 ['little', 'squirmish', 'but', 'much'],
 ['nuclear', 'powered', 'submarine', 'u.s.s'],
 ['jimmy',
  'carter',
  'on',
  'mission',
  'deep',
  'thick',
  'frigid',
  'ice',
  'near',
  'north',
  'pole',
  'it',
  'attacked',
  'giant',
  'super',
  'charged',
  'electric',
  'eels'],
 ['member',
  'the',
  'crew',
  'simmone',
  'jade',
  'mackinnon',
  'thinks',
  'has',
  'devised',
  'way',
  'communicate',
  'the',
  'monsters',
  'but',
  'not',
  'given',
  'much',
  'chance',
  'vague',
  'reasons'],
 ['also',
  'among',
  'crew',
  ':',
  'david',
  'keith',
  'mark',
  'sheppard',
  'sean',
  'whalen'],
 ['movie',
  'could',
  'been',
  'somewhat',
  'better',
  'the',
  'eels/monsters',
  'not',
  'cartoonish']]

Création d'une liste de StopWords et mots à supprimer de tous nos commentaires car inutiles !

### Tag des mots & chunking & chanking

L'ajout de tag aux mots est très important car il permet de leur donner un certain poid lors de l'analyse.

In [189]:
# Tag des mots sur le dataframe

from nltk import pos_tag
token = []
for i in range(0, len(dfc), 1):
    for j in range(0, len(dfc[i]), 1):
        token = pos_tag(dfc_phrases[i][j])
        dfc_phrases[i][j] = token


In [190]:
#nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [191]:
#def TagNLTK(df):
#    token = []
#    dfc = df.copy()
#    for i in range(0, len(df), 1):
#        for j in range(0, len(df[i]), 1):
#            token = pos_tag(df[i][j])
#            dfc[i][j] = token
#    return dfc

#TagNLTK(dfc_phrases)

In [194]:

import re

In [195]:
# Liste de tag qui sont à enlever 

NLTK_LIST = ['.','$','\'','(',')',':','CC','CD','DT','EX','FW','IN','LS','SYM','TO','WP',r'NPP.?','POS','PDT',r'PRP.?','RP','WTD','WP','WRB','"']



TREE_TAG_LIST = ['PUN:cit','PRP:det','PRO:rel','PRP:det','PRP', r'PRO', 'NUM','KON',r'DET']




In [196]:
# Nouveau df avec seulement les mots qui ne sont pas dans la liste

dfc_phrases_clear = dfc_phrases.copy()

display("Before")
print(dfc_phrases[10][2])


for i in range(0, len(dfc_phrases_clear), 1):
    for j in range(0, len(dfc_phrases_clear.loc[i]), 1):
        dfc_phrases_clear[i][j] = ([word for word,pos in dfc_phrases_clear[i][j] if (pos not in NLTK_LIST)])

display("After")
print(dfc_phrases_clear[10][2])


'Before'

[('however', 'RB'), ('many', 'JJ'), ('scenes', 'NNS'), ('were', 'VBD'), ('somehow', 'RB'), ('blocked', 'VBN'), ('the', 'DT'), ('bad', 'JJ'), ('scripting.the', 'NN'), ('reason', 'NN'), ('the', 'DT'), ('cool', 'NN'), ('idea', 'NN'), ('looking', 'VBG'), ('the', 'DT'), ('cyborg', 'NN'), ('which', 'WDT'), ('quite', 'RB'), ('different', 'JJ'), ('most', 'JJS'), ('such', 'JJ'), ('roles', 'NNS'), ("'ve", 'VBP'), ('seen', 'VBN'), ('far.everything', 'VBG'), ('else', 'RB'), ('this', 'DT'), ('movie', 'NN'), ('bad', 'JJ'), ('as', 'IN'), ('it', 'PRP'), ('be', 'VB')]


'After'

['however', 'many', 'scenes', 'were', 'somehow', 'blocked', 'bad', 'scripting.the', 'reason', 'cool', 'idea', 'looking', 'cyborg', 'which', 'quite', 'different', 'most', 'such', 'roles', "'ve", 'seen', 'far.everything', 'else', 'movie', 'bad', 'it', 'be']


**Suprression des mots trop rares, trop fréquents et stop words**


In [197]:
STOPWORDS = set(["a","about","above","after","again","against","ain","all","am","an","and","any","are","as","at","be","because","been","before","being","below","between","both","but","by","can","could","did","do","does","doesn","doing","down","during","each","few","for","from","further","had","has","have","having","he","her","here","hers","herself","him","himself","his","how","i","I","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","would","www","x","yes","yet","youd","youre","z","zero","a's","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder","do","am","were","be","been", "being", "have", "has","?","."])

In [198]:
dic_word = {} # Création d'un dictionnaire pour supprimer tous les mots qui n'aparaissent qu'une fois ou trop de fois
nb = 0 # Nombre de mot total

for i in range(0, len(dfc_phrases_clear), 1):
    for j in range(0, len(dfc_phrases_clear[i]), 1):
        for word in dfc_phrases_clear[i][j]:
            if word not in dic_word:
                dic_word[word] = 1
            else:
                dic_word[word] += 1
            nb+=1


display(dic_word)

{'obviously': 2,
 'made': 10,
 'show': 6,
 'famous': 4,
 'stripper': 1,
 'misty': 1,
 'ayers': 4,
 '``': 38,
 'acting': 11,
 "''": 38,
 'talents': 1,
 'bad': 7,
 'ca': 4,
 "n't": 31,
 'act.boring': 1,
 'little': 6,
 'tale': 3,
 'sweet': 1,
 'innocent': 1,
 'sally': 1,
 'being': 4,
 'drugged': 1,
 'forced': 1,
 'white': 1,
 'slavery': 1,
 'prostitution': 2,
 'she': 3,
 'meets': 1,
 'likable': 2,
 'tommy': 1,
 'cole': 1,
 'instantly': 1,
 'falls': 1,
 'her': 5,
 'wants': 4,
 'help': 2,
 'escape': 1,
 'can': 2,
 'really': 7,
 'wo': 5,
 'care.there': 1,
 'real': 6,
 'skin': 1,
 'strips': 1,
 'slowly': 1,
 'underwear': 1,
 'twice': 1,
 'rest': 3,
 'just': 8,
 'boring': 3,
 'chockful': 1,
 'atrocious': 2,
 'comedy': 2,
 'never': 7,
 'thought': 1,
 'funny': 3,
 'i': 21,
 'know': 7,
 'terrible': 1,
 'post-dubbed': 1,
 'dialogue': 1,
 'admit': 1,
 'was': 8,
 'end': 7,
 'did': 2,
 'see': 4,
 'coming': 1,
 "'s": 3,
 'enough': 1,
 'sit': 2,
 'also': 5,
 'attempts': 1,
 'hysterical': 1,
 'bomb': 1,

In [199]:
KEEPWORD = [] # Liste avec les mots à garder créée grâce au dico

for key, value in dic_word.items() :
    #display(key)
    if ((value > 1) and (value < (nb/25))) :
        KEEPWORD.append(key)
#display(KEEPWORD)

print("Taille dico avant " + str(len(dic_word)) + " et après " + str(len(KEEPWORD)))

Taille dico avant 1451 et après 382


In [200]:
# Petite fonction pour virer les raccourcis de langage
Rdico = {'y':'why', 'u':'you','sux':'suck','asap':'As soon as possible','jk':'joke', 'ur' : 'You are', 'r' : 'are'} 

def replaceW(word):
    for key,value in Rdico.items() :
        if word == key :
            return value
    return word

Maintenant on les vire de nos commentaires

In [201]:
dfc_clean = dfc_phrases_clear.copy()

print("Before")
print(dfc_clean[19])

for i in range(0, len(dfc_clean), 1):
    for j in range(0, len(dfc_clean.loc[i]), 1):
        dfc_clean[i][j] = ([replaceW(word) for word in dfc_clean[i][j] if ((word in KEEPWORD)and (word not in STOPWORDS))])
        # Nouveau df sans les stopword et avec seulement les mots définis à garder


print("After")
print(dfc_clean[19])

Before
[['mediocre', 'sci-fi', 'channel', 'original', 'picture'], ['little', 'squirmish', 'much'], ['nuclear', 'powered', 'submarine', 'u.s.s'], ['jimmy', 'carter', 'mission', 'deep', 'thick', 'frigid', 'ice', 'north', 'pole', 'it', 'attacked', 'giant', 'super', 'charged', 'electric', 'eels'], ['member', 'crew', 'simmone', 'jade', 'mackinnon', 'thinks', 'has', 'devised', 'way', 'communicate', 'monsters', 'not', 'given', 'much', 'chance', 'vague', 'reasons'], ['also', 'crew', 'david', 'keith', 'mark', 'sheppard', 'sean', 'whalen'], ['movie', 'could', 'been', 'somewhat', 'better', 'eels/monsters', 'not', 'cartoonish']]
After
[['original', 'picture'], [], [], ['charged'], ['member', 'crew', 'thinks', 'reasons'], ['crew', 'david', 'keith', 'mark', 'sean'], ['movie']]


Gestion de la négation

https://www.youtube.com/watch?v=88yOCX4ZyoA&list=PLcTXcpndN-Sl9eYrKM6jtcOTgC52EJnqH&index=17

### Relation entre les mots : N-Grams tokenizing

### Bag of Words

Une fois que nos data ont subit tous les pré-traitements, créer un nouveau fichier avec les données néttoyées pour permettre l'application de bag of Words. </br>
Peut être que les bag of words ne sont à appliquer que sur les mots les plus significatifs?

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
with open ("./Data/dataset.csv", "r") as file:
    X_train_counts = count_vect.fit_transform(file)
    

In [0]:
display(X_train_counts.shape)

In [0]:
display(X_train_counts.todense() )

In [0]:
from nltk.probability import FreqDist
fdist = FreqDist(tokens)

In [0]:
import matplotlib.pyplot as plt
fdist.plot(30,cumulative=False)
plt.show()

https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk